In [ ]:
# Setup plotting
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('animation', html='html5')

import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks

# Holding the dataset filepaths
line_file_path = 'temp'

# Reading the data
line_data = pd.read_csv(line_file_path)

# Making numpy arrays out of the datasets
line_numpy = np.array(line_data)

# Transposing the numpy arrays
line_numpy = line_numpy.T
design_numpy = design_numpy.T

# Splitting the arrays into training, validation, and test sets. 20%, 10%, 70% split
line_test, line_train = train_test_split(line_numpy, test_size = 0.2, train_size = 0.8) #can I just add shuffle = true to this?
line_valid, line_train = train_test_split(line_train, test_size = 0.125, train_size = 0.875)

# Standardizing the data
# find the standard deviation of each feature within the training set
sigma = np.std(line_train, axis=0)
# find the mean of each feature within the training set
mu = np.mean(line_train, axis=0)
# standardize our data based on mu and sigma from the training set
standard_line_train = (line_train - mu) / sigma
standard_line_valid = (line_valid - mu) / sigma
standard_line_test = (line_test - mu) / sigma

# Creating the neural network
input_shape = [standard_line_train.shape[1]]
model = keras.Sequential([
    layers.BatchNormalization(input_shape=input_shape),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.4),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.4),
    layers.BatchNormalization(),
    layers.Dense(1)
])

model.compile(
    optimizer='adam',
    loss='mae',
)
history = model.fit(
    standard_line_train,
    validation_data=(standard_line_valid),
    batch_size=512,
    epochs=150,
    verbose=0,
)
history_df = pd.DataFrame(history.history)
history_df.loc[0:, ['loss', 'val_loss']].plot()
print("Minimum Absolute Error: {:0.4f}".format(history_df['val_loss'].min()))